# Knowit julekalender 2020

In [15]:
import re
import requests
import time

import numpy as np
import pandas as pd

In [8]:
luke1 = requests.get('https://julekalender-backend.knowit.no/challenges/1/attachments/numbers.txt').text
tall_fra_fil = set([int(tall) for tall in luke1.split(sep=',')])
alle_tall = set([t for t in range(min(tall_fra_fil), max(tall_fra_fil) + 1)])
print(f'Tallet som mangler i luke 1 er {alle_tall.difference(tall_fra_fil)}')

Tallet som mangler i luke 1 er {81273}


In [10]:
def is_prime(n):
    for check in range(3, n, 1):
        if n%check==0:
            return False
    return True

def previous_prime(n):
    for check in range(n, 1, -1):
        if is_prime(check):
            return check
    return n

deliveries = set()
package_no = 0
while package_no < 5433000:
    if '7' in str(package_no):
        p_prime = previous_prime(package_no)
        next_package = package_no + p_prime + 1
        #print(f'Jeg hater tallet {package_no}! Forrige primtall er {p_prime}. Neste pakke er {next_package}')
        package_no = next_package
    else:
        deliveries.add(package_no)
        package_no += 1
print(f'Leverte {len(deliveries)} pakker:\n {sorted(deliveries)}')

Leverte 69 pakker:
 [0, 1, 2, 3, 4, 5, 6, 15, 16, 35, 36, 149, 150, 151, 152, 153, 154, 155, 156, 315, 316, 635, 636, 1269, 2530, 2531, 2532, 2533, 2534, 2535, 2536, 5069, 10130, 10131, 10132, 10133, 10134, 10135, 10136, 40541, 40542, 40543, 40544, 40545, 40546, 81091, 81092, 81093, 81094, 81095, 81096, 162195, 162196, 324341, 324342, 324343, 324344, 324345, 324346, 648689, 648690, 648691, 648692, 648693, 648694, 648695, 648696, 5189505, 5189506]


In [12]:
matrise_url = 'https://gist.githubusercontent.com/knowitkodekalender/d277d4f01a9fe10f7c1d92e2d17f1b31/raw/49da54e4372a83f4fc11d7137f19fc8b4c58bda6/matrix.txt'
fasitliste_url = 'https://gist.githubusercontent.com/knowitkodekalender/9e1ba20cd879b0c6d7af4ccfe8a87a19/raw/b19ae9548a33a825e2275d0283986070b9b7a126/wordlist.txt'
fasitliste = requests.get(fasitliste_url).text.split(sep='\n')
matrise = requests.get(matrise_url).text.split(sep='\n')[:-1]
search_directions = {'🡬': (-1, -1), '🡩': (-1, 0),
                     '🡭': (-1, 1), '🡨': (0, -1),
                     '🡪': (0, 1), '🡯': (1, -1),
                     '🡫': (1, 0), '🡮': (1, 1)}

def search_direction(search_string, x, y, direction):
    offset_x, offset_y = direction
    search_x, search_y = x + offset_x, y + offset_y
    try:
        if search_string[0] == matrise[search_x][search_y] and search_x >= 0 and search_y >= 0:
            if len(search_string) == 1:
                return search_x, search_y
            else:
                return search_direction(search_string[1:], search_x, search_y, direction)
    except:
        pass
    return None

found_dict = dict()
for search_word in fasitliste:
    for x, row in enumerate(matrise):
        for y, char in enumerate(row):
                if matrise[x][y] == search_word[0]:
                    for arrow, direction in search_directions.items():
                        try:
                            end_x, end_y = search_direction(search_word[1:], x, y, direction)
                            found_dict[search_word] = ((x, y), (end_x, end_y), arrow)
                        except:
                            pass

print('Fant følgende:')
for k, v in found_dict.items():    
    print(f'{k:<20}{v[0]} {v[2]} {v[1]}')
print(f'Fant ikke {[bom for bom in fasitliste if bom not in found_dict.keys()]}')

Fant følgende:
nisseverksted       (390, 480) 🡭 (378, 492)
pepperkake          (252, 858) 🡬 (243, 849)
adventskalender     (566, 767) 🡮 (580, 781)
klementin           (369, 762) 🡭 (361, 770)
krampus             (113, 135) 🡩 (107, 135)
juletre             (476, 690) 🡭 (470, 696)
julestjerne         (657, 798) 🡪 (657, 808)
gløggkos            (220, 501) 🡮 (227, 508)
mandel              (581, 355) 🡪 (581, 360)
sledespor           (836, 160) 🡫 (844, 160)
nordpolen           (341, 707) 🡨 (341, 699)
nellik              (187, 880) 🡨 (187, 875)
svineribbe          (498, 299) 🡮 (507, 308)
lutefisk            (495, 187) 🡩 (488, 187)
medisterkake        (290, 866) 🡩 (279, 866)
grevinne            (540, 703) 🡫 (547, 703)
hovmester           (873, 356) 🡫 (881, 356)
sølvgutt            (524, 124) 🡯 (531, 117)
jesusbarnet         (734, 865) 🡨 (734, 855)
julestrømpe         (192, 439) 🡬 (182, 429)
rudolf              (607, 262) 🡬 (602, 257)
akevitt             (292, 486) 🡪 (292, 492)
Fant ikke ['marsi

In [14]:
leveringsliste_url = 'https://julekalender-backend.knowit.no/challenges/4/attachments/leveringsliste.txt'
leveringsliste = requests.get(leveringsliste_url).text
ingredienser = {'sukker': 2, 'mel': 3, 'melk': 3, 'egg': 1}
leveranser = {}
ant_kaker = {}
for i in ingredienser.keys():
    leveranser[i] = sum([int(s) for s in re.findall(f'{i}: (\d*)', leveringsliste)])
    ant_kaker[i] = leveranser[i]/ingredienser[i]
    print(f'{leveranser[i]} enehter {i} holder til {ant_kaker[i]:.2f} kaker')
print(f'Vi kan bake {min([int(a) for a in ant_kaker.values()])} kaker')

4565133 enehter sukker holder til 2282566.50 kaker
4374042 enehter mel holder til 1458014.00 kaker
4489966 enehter melk holder til 1496655.33 kaker
4711270 enehter egg holder til 4711270.00 kaker
Vi kan bake 1458014 kaker


In [17]:
instruksjoner_url = 'https://julekalender-backend.knowit.no/challenges/5/attachments/rute.txt'
instruksjoner = requests.get(instruksjoner_url).text
retninger = {'V': (-1, 0), 'H': (1, 0), 'O': (0, 1), 'N': (0, -1)}

x, y = [0], [0]
for instruksjon in instruksjoner:
    x.append(x[-1] + retninger[instruksjon][0])
    y.append(y[-1] + retninger[instruksjon][1])
    
areal = 0
for i in range(1, len(instruksjoner) + 1):
    areal += (x[i - 1] + x[i]) * (y[i - 1] - y[i])
print(f'Arealet er {abs(areal/2)}')

Arealet er 118000.0


In [23]:
pakker_url = 'https://julekalender-backend.knowit.no/challenges/6/attachments/godteri.txt'
pakker = [int(s) for s in requests.get(pakker_url).text.split(',')]
godteri_akkumulert = np.array(pakker).cumsum()
ant_alver = 127
print(f'Det blir {max(godteri_akkumulert[godteri_akkumulert % ant_alver == 0]) / ant_alver} godteri på hver.')

Det blir 952.0 godteri på hver.


In [26]:
symmetre_url = 'https://julekalender-backend.knowit.no/challenges/7/attachments/forest.txt'
symmetre_rader = requests.get(symmetre_url).text.split('\n')[:-1]

trær = [[]]
for kolonne in map(list, zip(*symmetre_rader)):
    if kolonne.count('#') == 0 and trær[-1]:
        trær.append([])
    else:
        trær[-1].append(''.join(kolonne))

alle_trær = list(filter(None, trær))
symmetriske_trær = sum([tre[::1] == tre[::-1] for tre in alle_trær])
print(f'Vi har {len(alle_trær)} trær til sammen hvorav {symmetriske_trær} er symmetriske.')

Vi har 8193 trær til sammen hvorav 1 er symmetriske.


In [27]:
inputfil_url = 'https://julekalender-backend.knowit.no/challenges/8/attachments/input.txt'
rådata = requests.get(inputfil_url).text.split('\n')[:-1]

lokasjoner_raw = [re.findall(r'(.*): (.*)', r)[0] for r in rådata[:50]]
lokasjoner = {lokasjon[0]: eval(lokasjon[1]) for lokasjon in lokasjoner_raw}
reiserute = list(map(lokasjoner.get, rådata[50:]))

def get_retning(fra, til):
    if til[0] > fra[0]:
        return (1, 0)
    elif til[0] < fra[0]:
        return (-1, 0)
    elif til[1] > fra[1]:
        return (0, 1)
    elif til[1] < fra[1]:
        return (0, -1)

lokasjon_x, lokasjon_y = np.array([v[0] for v in lokasjoner.values()]), np.array([v[1] for v in lokasjoner.values()])
x, y = [0], [0]

for destinasjon in reiserute:
    while (x[-1], y[-1]) != destinasjon:
        retning = get_retning((x[-1], y[-1]), destinasjon)
        x.append(x[-1] + retning[0])
        y.append(y[-1] + retning[1])

x.pop(0) # Remove start position
y.pop(0)

distances = abs(np.subtract(np.array(x), lokasjon_x[:,None])) + abs(np.subtract(np.array(y), lokasjon_y[:,None]))
time_increment_brackets = [distances == 0, (distances > 0) & (distances < 5), (distances >= 5) & (distances < 20), (distances >= 20) & (distances < 50)]
time_increments = [0, 0.25, 0.5, 0.75]
total_times = np.select(time_increment_brackets, time_increments, default=1).sum(axis=1)

print(f'Differansen er {total_times.max() - total_times.min()}')

Differansen er 6845.25


In [28]:
rådata = requests.get('https://julekalender-backend.knowit.no/challenges/9/attachments/elves.txt').text.split('\n')[:-1]
start_time = time.time()
syk = np.array(np.array(list([*s] for s in rådata)) == 'S', dtype=np.int8) # int array
antall_syke = [syk.sum()]

while True:
    syke_rundt = np.pad(syk, 1)[1:-1, 2:] + np.pad(syk, 1)[1:-1, :-2] + np.pad(syk, 1)[2:, 1:-1] + np.pad(syk, 1)[:-2, 1:-1]
    syk = syk | np.array(syke_rundt >= 2)
    
    if syk.sum() == antall_syke[-1]:
        break # ingen nye syke
    else:
        antall_syke.append(syk.sum())

print(f'Det tar {len(antall_syke)} dager til smitten stopper opp. Kjøretid: {time.time() - start_time:.3f}s.')

Det tar 158 dager til smitten stopper opp. Kjøretid: 0.062s.


In [29]:
resultatliste_url = 'https://julekalender-backend.knowit.no/challenges/10/attachments/leker.txt'
resultatliste = requests.get(resultatliste_url).text.split('\n')[:-1]

datasett = pd.DataFrame([{'spiller': s, 'øvelse': i, 'pos': j + 1, 'tot_deltagere': 1 + rad.count(',')} for i, rad in enumerate(resultatliste) for j, s in enumerate(rad.split(','))])
datasett['poeng'] = datasett['tot_deltagere'] - datasett['pos']
datasett.groupby('spiller')['poeng'].sum().sort_values(ascending=False)

spiller
ti    7776
gk    7249
vs    7235
ge    7192
hn    7119
      ... 
ua    3123
vb    2989
dj    2893
ox    2724
el    2221
Name: poeng, Length: 676, dtype: int64

In [30]:
kandidat_hint = requests.get('https://julekalender-backend.knowit.no/challenges/11/attachments/hint.txt').text.split('\n')[:-1]

def get_rader(hint):
    shifted = '' if len(hint) <= 1 else ''.join([chr(ord('a') + (ord(o) + ord(w) + 1 - 2*ord('a'))%26) for w, o in zip(hint, hint[1:])])
    return [shifted] if len(shifted) <= 1 else [shifted] + get_rader(shifted)

def get_kolonner(hint):
    rader_padded = [hint] + [x.ljust(len(hint)) for x in get_rader(hint)]
    return map(''.join, zip(*rader_padded))

print([(kandidat, kolonne) for kandidat in kandidat_hint for kolonne in get_kolonner(kandidat) if 'eamqia' in kolonne])

[('metropolitan', 'tlseamqiay  ')]


In [33]:
rådata = np.array(list(requests.get('https://julekalender-backend.knowit.no/challenges/12/attachments/family.txt').text))
nivå = np.select([rådata == '(', rådata == ')'], [1, -1], 0).cumsum()
navn_pr_nivå_df = pd.DataFrame({'er_navn': np.char.isupper(rådata), 'nivå': nivå}).groupby('nivå')['er_navn'].sum()
print(f'Nivået med flest alver har {navn_pr_nivå_df.max()}.')

Nivået med flest alver har 5965.


In [35]:
strengen = requests.get('https://julekalender-backend.knowit.no/challenges/13/attachments/text.txt').text

start_time = time.time()
for i, a in enumerate('abcdefghijklmnopqrstuvwxyz'):
    if strengen.count(a) >= i+1:
        strengen = ''.join(strengen.split(a)[:i+1] + [a] + strengen.split(a)[i+1:])
    else:
        strengen = strengen.replace(a, '')
print(f'Svaret er {strengen}')
print(f'Kjørte på {time.time() - start_time:.7f} sekunder.')

Svaret er aebdhgcfijmqolpnvutkzsxryw 
Kjørte på 0.0000000 sekunder.


In [37]:
from sympy import sieve
start_time = time.time()

LST_LEN = 1800813
li = [0, 1]
li_set = set(li)
for n in range(2, LST_LEN):
    if li[-2] - n >= 0 and li[-2] - n not in li_set:
        li.append(li[-2] - n)
    else:
        li.append(li[-2] + n)
    li_set.add(li[-1])

print(f'Tid før primtallsberegning: {time.time() - start_time:.3f}')
    
primes = set(sieve.primerange(0, max(li_set)))

print(f'Antall primtall er {len([x for x in li if x in primes])}')
print(f'Total tid {time.time() - start_time:.3f}')

Tid før primtallsberegning: 1.032
Antall primtall er 67321
Total tid 1.359


In [38]:
ordbok = set(requests.get('https://julekalender-backend.knowit.no/challenges/15/attachments/wordlist.txt').text.split('\n'))
ordparliste = [tuple(ordpar.split(', ')) for ordpar in requests.get('https://julekalender-backend.knowit.no/challenges/15/attachments/riddles.txt').text.split('\n')[:-1]]
alle_limord = set()
for ord_1, ord_2 in ordparliste:
    starter_med_ord_1 = set([ordbokord.replace(ord_1, '') for ordbokord in ordbok if ordbokord.startswith(ord_1)])
    slutter_med_ord_2 = set([ordbokord.replace(ord_2, '') for ordbokord in ordbok if ordbokord.endswith(ord_2)])
    alle_limord.update(starter_med_ord_1.intersection(slutter_med_ord_2).intersection(ordbok))
print(f'Totalt antall bokstaver: {sum([len(limord) for limord in alle_limord])}')

Totalt antall bokstaver: 278


In [3]:
# sympy versjon
import numpy as np
from sympy import divisors

x = np.arange(1000000) + 1
d_sums = np.array([sum(divisors(i)) for i in x])
diff = np.clip(d_sums - 2 * x, 0, None)
print(f'Løsning = {np.isin(np.sqrt(diff), x).sum()}')

Løsning = 705


In [4]:
# numpy versjon
import time
import numpy as np
start_time = time.time()
n = 1000000
x = np.arange(1, n + 1).astype(np.uint32)
pot_divs = np.arange(1, int(np.sqrt(n) + 1)).astype(np.uint32)
x_flip = x[:, None]
is_divisor = (x_flip % pot_divs == 0) & (pot_divs <= np.sqrt(x_flip))
is_sqrt = x_flip == pot_divs ** 2
div_sums = np.sum(np.where(is_divisor, pot_divs, 0) + np.where((is_divisor) & (~is_sqrt), x_flip // pot_divs, 0), axis=1)
d = np.clip(div_sums - 2 * x, 0, None)
print(f'Svar: {np.isin(np.sqrt(d), x).sum()}')
print(f'{time.time() - start_time}')

Svar: 705
21.194302320480347


In [1]:
import requests
from scipy.signal import convolve
import numpy as np

rådata = requests.get('https://julekalender-backend.knowit.no/challenges/17/attachments/kart.txt').text.split('\n')[:-1]

støvsuger = np.array([[0, 0, 1, 1, 1, 0, 0],
                     [0, 1, 1, 1, 1, 1, 0],
                     [1, 1, 1, 1, 1, 1, 1],
                     [1, 1, 1, 1, 1, 1, 1],
                     [1, 1, 1, 1, 1, 1, 1],
                     [0, 1, 1, 1, 1, 1, 0],
                     [0, 0, 1, 1, 1, 0, 0]])

børster = np.array([[1, 1, 1, 0, 0, 0, 1, 1, 1],
                    [1, 1, 1, 1, 1, 1, 1, 1, 1],
                    [1, 1, 1, 1, 1, 1, 1, 1, 1],
                    [0, 1, 1, 1, 1, 1, 1, 1, 0],
                    [0, 1, 1, 1, 1, 1, 1, 1, 0],
                    [0, 1, 1, 1, 1, 1, 1, 1, 0],
                    [1, 1, 1, 1, 1, 1, 1, 1, 1],
                    [1, 1, 1, 1, 1, 1, 1, 1, 1],
                    [1, 1, 1, 0, 0, 0, 1, 1, 1]])

gulv = np.pad(np.array(np.array(list([*s] for s in rådata)) != 'x'), 5)
støv_pos = convolve(gulv, støvsuger, mode='same') == støvsuger.sum()
bs = np.array([np.roll(np.roll(støv_pos, ix[0] - 4, axis=1), ix[1] - 4, axis=0) for ix, b in np.ndenumerate(børster) if b == 1])
print(f'Skitne overflater: {gulv.sum() - np.array(np.any(bs, axis=0) & gulv).sum()}')

Skitne overflater: 288903


In [ ]:
ordliste = set(requests.get('https://julekalender-backend.knowit.no/challenges/18/attachments/wordlist.txt').text.split('\n'))

def er_palindrom(s):
    return s == s[::-1]

def er_palinestendrom(s):
    
    def rekursiv_del(s):
        if len(s) <= 2 or er_palindrom(s):
            return True
        elif sorted(s[0:2]) == sorted(s[-2:]):
            return rekursiv_del(s[2:-2])
        elif s[0] == s[-1]:
            return rekursiv_del(s[1:-1])
        else:
            return False
    
    if len(s) <= 3 or er_palindrom(s):
        return False
    else:
        return rekursiv_del(s)
    
palinestendrom = [np for np in ordliste if er_palinestendrom(np)]

print(f'Vi har {len(palinestendrom)} palinestendrom.')

In [5]:
import requests
from collections import Counter

def stollek(regel, pos, deltagere):
    i = 0
    while len(deltagere) > 1:
        deltagere = [deltagere[(i - pos) % len(deltagere)] for i, x in enumerate(deltagere)]
        if regel == 1:
            deltagere.pop(0)
        if regel == 2:
            if i >= len(deltagere):
                i = 0
            deltagere.pop(i)
            i += 1
        if regel == 3:
            if len(deltagere) == 2:
                deltagere.pop(0)
            elif len(deltagere) % 2 == 0:
                deltagere = deltagere[:len(deltagere)//2 - 1] + deltagere[len(deltagere)//2 + 1:]
            else:
                deltagere.pop(len(deltagere)//2)
        elif regel == 4:
            deltagere.pop(-1)
    return deltagere[0]

rådata = [s.strip('[], ') for s in requests.get('https://julekalender-backend.knowit.no/challenges/19/attachments/input.txt').text.split('\n')]

stolleker = []
for linje in rådata:
    linje_deler = linje.split(' ')
    stolleker.append({'regel': int(linje_deler.pop(0)), 'rotasjon': int(linje_deler.pop(0)), 'deltagere': linje_deler})

vinnere = Counter([stollek(lek['regel'], lek['rotasjon'], lek['deltagere']) for lek in stolleker])
print(f'Vinneren er {vinnere.most_common(1)[0][0]} med {vinnere.most_common(1)[0][1]} seiere')

Vinneren er Spencer, med 10 seiere


In [3]:
import requests

battle = [t.split(': ') for t in requests.get('https://julekalender-backend.knowit.no/challenges/23/attachments/rap_battle.txt').text.split('\n')[:-1]]
base_ord = dict(t.split(' ') for t in requests.get('https://julekalender-backend.knowit.no/challenges/23/attachments/basewords.txt').text.split('\n')[:-1])

def vokaler(ordz):
    return sum(1 for t in ordz if t in 'aeiouyæøå')

def rep_div(ordzliste):
    i = 1
    grunnord = ordzliste.pop().replace('jule', '')
    while ordzliste and ordzliste.pop().replace('jule', '') == grunnord:
        i += 1
    return i

def score(rap_linje):
    poeng = 0
    rappe_ordz = rap_linje.split(' ')
    for i, ordz in enumerate(rappe_ordz):
        grunnverdi = int(base_ord.get(ordz.replace('jule', ''), 0))
        vokalbonus = (int(ordz.startswith('jule')) + 1) * max((vokaler(ordz) - vokaler(rappe_ordz[max(i-1, 0)])), 0)
        poeng += (grunnverdi + vokalbonus) // rep_div(rappe_ordz[:i + 1])
    return poeng

nizzy_g = sum(score(b[1]) for b in battle if b[0] == 'Nizzy G')
lil_niz_x = sum(score(b[1]) for b in battle if b[0] == 'Lil Niz X')

print(f'Nizzy G får {nizzy_g} poeng, Lil Niz X får {lil_niz_x}')

Nizzy G får 3316 poeng, Lil Niz X får 2919


In [2]:
import requests, numpy as np
fuel = np.array(list(requests.get('https://julekalender-backend.knowit.no/challenges/24/attachments/rute.txt').text), dtype=np.int).cumsum() * 2 + 10
print(f'Nissen orker å besøke {np.argmax(fuel < np.arange(1, len(fuel) + 1))} hus. God jul🎄')

Nissen orker å besøke 3333490 hus. God jul🎄


In [5]:
' '.__ne__

<method-wrapper '__ne__' of str object at 0x000002BDF15E2EF0>